In [ ]:
# 卷积神经网络的手写实现
import torch
import torch.nn as nn

def conv_example(in_channel, kernel):
    # in_channel (28, 28)
    # kernel (5, 5)
    output = torch.zeros(24, 24)
    for h in range(24):
        for w in range(24):
            inputs = in_channel[h:h+5, w:w+5]
            output[h, w] = (inputs * kernel).sum()
    return output

In [ ]:
x = torch.randn(1, 1, 28, 28)
m = nn.Conv2d(1, 1, (5, 5), bias = False)
result = conv_example(x.squeeze(0, 1), m.weight.squeeze(0, 1))
result.shape

In [ ]:
# 卷积神经网络的实现
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

torch.manual_seed(726)

# 加载数据
dataset = datasets.MNIST(".\mnist", train=True, download=True, transform=transforms.ToTensor())
train_set, val_set = random_split(dataset, [50000, 10000])
test_set = datasets.MNIST('.\mnist', train=False, download=False, transform=transforms.ToTensor())

# 创建数据加载器
train_loader = DataLoader(train_set, batch_size=500, shuffle=True)
val_loader = DataLoader(val_set, batch_size=500, shuffle=True)
test_loader = DataLoader(test_set, batch_size=500, shuffle=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, (5, 5))
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 40, (5, 5))
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(40 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        # x :(B, 1, 28, 28)
        B = x.shape[0]
        x = F.relu(self.conv1(x))  # (B, 20, 24, 24)
        x = self.pool1(x)          # (B, 20, 12, 12)
        x = F.relu(self.conv2(x))  # (B, 40, 8, 8)
        x = self.pool2(x)          # (B, 40, 4, 4)
        x = F.relu(self.fc1(x.view(B, -1)))  # (B, 120)
        x = self.fc2(x)                      # (B, 10)
        return x

model = CNN()


In [ ]:
# 评估模型

# 评估轮数，取平均
eval_iters = 10

# 评估训练集，验证集和测试集的loss和精度
def estimate_loss(model):
    re = {}
    # 将模型切换为评估模式
    model.eval()
    re['train'] = _loss(model, train_loader)
    re['val'] = _loss(model, val_loader)
    re['test'] = _loss(model, test_loader)
    # 将模型切换为训练模式
    model.train()
    return re

def _loss(model, dataloader):
    # 估算模型效果
    loss = []
    acc = []
    data_iter = iter(dataloader)
    for t in range(eval_iters):
        inputs, labels = next(data_iter) # inputs:(500, 1, 28, 28) label:(500)
        B = inputs.shape[0]
        logits = model(inputs) # logits: (500, 10)
        loss.append(F.cross_entropy(logits, labels))
        preds = torch.argmax(logits, dim=-1)
        acc.append((preds == labels).sum() / B)
    re = {
        'loss' : torch.tensor(loss).mean().item(),
        'acc': torch.tensor(acc).mean().item()
    }
    return re




In [ ]:
def train_model(model, optimizer, epoch=10):
    for e in range(epoch):
        for data in train_loader:
            inputs, labels = data
            logits = model(inputs)
            loss = F.cross_entropy(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # 每个epoch之后，在训练集、验证集和测试集上查看损失
        stats = estimate_loss(model)
        train_loss = f'{stats["train"]["loss"]:.3f}'
        val_loss = f'{stats["val"]["loss"]:.3f}'
        test_loss = f'{stats["test"]["loss"]:.3f}'
        print(f'epoch {e} train {train_loss} val {val_loss} test {test_loss}')


In [ ]:
train_model(model, optim.Adam(model.parameters(), lr=0.01))

In [ ]:
estimate_loss(model)

In [ ]:
# 卷积神经网络优化 -> 加入层归一化层和随机失活层
class CNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, (5, 5))
        self.ln1 = nn.LayerNorm([20, 24, 24])
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 40, (5, 5))
        self.ln2 = nn.LayerNorm([40, 8, 8])
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(40 * 4 * 4, 120)
        self.dp = nn.Dropout(0.2)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        B = x.shape[0]
        x = F.relu(self.ln1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.ln2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.fc1(x.view(B, -1)))
        x = self.dp(x)
        x = self.fc2(x)
        return x

model2 = CNN2()


In [ ]:
train_model(model2, optimizer=optim.Adam(model2.parameters(), lr=0.01))